In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from datetime import datetime
import statsmodels.api as sm

# Use Kamppi as an example, could be generalized to all stations
df = pd.read_csv("datasets/Jämeräntaival_hourly_aggregate.csv")
df['Departure'] = pd.to_datetime(df['Departure'], format='mixed')
df.set_index('Departure', inplace=True)

train_end = datetime(year=2022, month=10, day=31)
train_start = datetime(year=2018,month=4,day=1)
train_data = df[:train_end]
test_end = datetime(year=2023, month=10, day = 31)
test_data  = df[datetime(year=2023,month=4,day=1):]

parameter_space = list()

for i in range(1,4):
    for j in range(1,4):
        for k in range(1,4):
            parameter_space.append((i,j,k))


param_mse = list()
for param in parameter_space:
    model = sm.tsa.statespace.SARIMAX(train_data['trip'], order=param, seasonal_order=(0, 1, 0, 24), freq='h').fit()
    forecast = model.forecast(datetime(year=2023,month=10,day=31,hour=17))
    forecast = forecast[datetime(year=2023,month=4,day=1):]
    #test_data.index = pd.DatetimeIndex(forecast.index)
    
    final = pd.concat([forecast,test_data], axis=1)
    errors = final['trip'] - final['predicted_mean']
    mse = np.mean(errors**2)

    param_mse.append((mse, param))

'''

[(1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 2, 1), (1, 2, 2), (1, 2, 3), (1, 3, 1), (1, 3, 2), (1, 3, 3), (2, 1, 1), (2, 1, 2), (2, 1, 3), (2, 2, 1), (2, 2, 2), (2, 2, 3), (2, 3, 1), (2, 3, 2), (2, 3, 3), (3, 1, 1), (3, 1, 2), (3, 1, 3), (3, 2, 1), (3, 2, 2), (3, 2, 3), (3, 3, 1), (3, 3, 2), (3, 3, 3)]
27


"\nparam_mse = list()\nfor param in parameter_space:\n    model = sm.tsa.statespace.SARIMAX(train_data['trip'], order=param, seasonal_order=(0, 1, 0, 24), freq='h').fit()\n    forecast = model.forecast(datetime(year=2023,month=10,day=31,hour=17))\n    forecast = forecast[datetime(year=2023,month=4,day=1):]\n    #test_data.index = pd.DatetimeIndex(forecast.index)\n    \n    final = pd.concat([forecast,test_data], axis=1)\n    errors = final['trip'] - final['predicted_mean']\n    mse = np.mean(errors**2)\n\n    param_mse.append((mse, param))\n\n"

In [48]:
forecast = model.forecast(datetime(year=2023,month=10,day=31,hour=17))
forecast = forecast[datetime(year=2023,month=4,day=1):]
#test_data.index = pd.DatetimeIndex(forecast.index)

final = pd.concat([forecast,test_data], axis=1)
errors = final['trip'] - final['predicted_mean']
print(errors)
mse = np.mean(errors**2)

print(mse)

2023-04-01 00:00:00    0.223308
2023-04-01 01:00:00   -0.336395
2023-04-01 02:00:00   -1.678867
2023-04-01 03:00:00   -1.754065
2023-04-01 04:00:00    0.229423
                         ...   
2023-10-31 13:00:00    0.537703
2023-10-31 14:00:00   -0.462297
2023-10-31 15:00:00   -0.462297
2023-10-31 16:00:00    0.537703
2023-10-31 17:00:00    0.537703
Freq: h, Length: 5130, dtype: float64
10.388099588499935
